### Word ranking

#### Task description
Create your own WordCount program and process Wikipedia dump. Use the second job to sort words by quantity in the reverse order (most popular first). Output format:

word < tab > count

The result is the 7th word by popularity and its quantity.

In [44]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [45]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [46]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [47]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [48]:
%%writefile mapper-words-ranking.py

import sys

for line in sys.stdin:
    try:
        w, c = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    print "%d\t%s" % (int(c), w)

Overwriting mapper-words-ranking.py


In [49]:
%%writefile reducer-words-ranking.py

import sys

for line in sys.stdin:
    try:
        c, w = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    print "%s\t%s" % (w, c)

Writing reducer-words-ranking.py


In [54]:
%%bash

OUT_DIR_1="wordcount_result_lkv"
NUM_REDUCERS_1=8

OUTPUT_2="words_ranking_result_lkv"
NUM_REDUCERS_2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUTPUT_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_1} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_1} > /dev/null
    

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
     -D mapred.jab.name="Streaming word ranking" \
     -D mapreduce.job.reduces=${NUM_REDUCERS_2} \
     -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
     -D mapred.text.key.comparator.options=-nr\
     -files mapper-words-ranking.py,reducer-words-ranking.py \
     -mapper "python mapper-words-ranking.py" \
     -reducer "python reducer-words-ranking.py" \
     -input wordcount_result_lkv/part* \
     -output ${OUTPUT_2} > /dev/null

hdfs dfs -cat ${OUTPUT_2}/part-00000 | head -7 | tail -1

is	126420


rm: `words_ranking_result_lkv': No such file or directory
18/01/14 11:19:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 11:19:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 11:19:53 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/14 11:19:53 INFO mapreduce.JobSubmitter: number of splits:2
18/01/14 11:19:53 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1515878426045_0008
18/01/14 11:19:54 INFO impl.YarnClientImpl: Submitted application application_1515878426045_0008
18/01/14 11:19:54 INFO mapreduce.Job: The url to track the job: http://330ad0c23dc0:8088/proxy/application_1515878426045_0008/
18/01/14 11:19:54 INFO mapreduce.Job: Running job: job_1515878426045_0008
18/01/14 11:20:00 INFO mapreduce.Job: Job job_1515878426045_0008 running in uber mode : false
18/01/14 11:20:00 INFO mapreduce.Job:  map 0% reduce 0%
18/01/14 11:20:16 INFO mapreduce.Job:  map 28% reduce 0%
18/01/14 11:20:22 INFO